In [1]:
import pandas as pd
import numpy as np
import os, json

In [2]:
DATAPATH = 'data'
VERSION = 'cord19_v20'
META_PATH ="/".join([DATAPATH, VERSION, 'metadata.csv'])

In [3]:
meta_df = pd.read_csv(META_PATH)
print(meta_df.shape)
# meta_df.head()

(63571, 18)


C:\Users\Khachatur\Miniconda3\envs\tensorflow\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,cord_uid,sha,source_x,title,doi,pmcid,pubmed_id,license,abstract,publish_time,authors,journal,mag_id,who_covidence_id,arxiv_id,pdf_json_files,pmc_json_files,url
0,zjufx4fo,b2897e1277f56641193a6db73825f707eed3e4c9,PMC,Sequence requirements for RNA strand transfer ...,10.1093/emboj/20.24.7220,PMC125340,11742998.0,green-oa,Nidovirus subgenomic mRNAs contain a leader se...,2001-12-17,"Pasternak, Alexander O.; van den Born, Erwin; ...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/b2897e1277f56641193a6...,document_parses/pmc_json/PMC125340.xml.json,http://europepmc.org/articles/pmc125340?pdf=re...
1,ymceytj3,e3d0d482ebd9a8ba81c254cc433f314142e72174,PMC,"Crystal structure of murine sCEACAM1a[1,4]: a ...",10.1093/emboj/21.9.2076,PMC125375,11980704.0,green-oa,CEACAM1 is a member of the carcinoembryonic an...,2002-05-01,"Tan, Kemin; Zelus, Bruce D.; Meijers, Rob; Liu...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/e3d0d482ebd9a8ba81c25...,document_parses/pmc_json/PMC125375.xml.json,http://europepmc.org/articles/pmc125375?pdf=re...
2,wzj2glte,00b1d99e70f779eb4ede50059db469c65e8c1469,PMC,Synthesis of a novel hepatitis C virus protein...,10.1093/emboj/20.14.3840,PMC125543,11447125.0,no-cc,Hepatitis C virus (HCV) is an important human ...,2001-07-16,"Xu, Zhenming; Choi, Jinah; Yen, T.S.Benedict; ...",EMBO J,NaN,NaN,NaN,document_parses/pdf_json/00b1d99e70f779eb4ede5...,document_parses/pmc_json/PMC125543.xml.json,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...
3,2sfqsfm1,cf584e00f637cbd8f1bb35f3f09f5ed07b71aeb0,PMC,Structure of coronavirus main proteinase revea...,10.1093/emboj/cdf327,PMC126080,12093723.0,green-oa,The key enzyme in coronavirus polyprotein proc...,2002-07-01,"Anand, Kanchan; Palm, Gottfried J.; Mesters, J...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/cf584e00f637cbd8f1bb3...,document_parses/pmc_json/PMC126080.xml.json,http://europepmc.org/articles/pmc126080?pdf=re...
4,i0zym7iq,dde02f11923815e6a16a31dd6298c46b109c5dfa,PMC,Discontinuous and non-discontinuous subgenomic...,10.1093/emboj/cdf635,PMC136939,12456663.0,green-oa,"Arteri-, corona-, toro- and roniviruses are ev...",2002-12-01,"van Vliet, A.L.W.; Smits, S.L.; Rottier, P.J.M...",The EMBO Journal,NaN,NaN,NaN,document_parses/pdf_json/dde02f11923815e6a16a3...,document_parses/pmc_json/PMC136939.xml.json,http://europepmc.org/articles/pmc136939?pdf=re...


In [38]:
cuid_dup = meta_df[meta_df.duplicated(['cord_uid'])]
cuid_dup_set = set(cuid_dup['cord_uid'])
print("All cuid duplicate records:"len(cuid_dup))

All cuid duplicate records: 44


In [9]:
for uid in cuid_dup_set:
    dup_df = meta_df[meta_df['cord_uid'] == uid]
    if len(dup_df['pubmed_id'].unique()) != 1 and len(dup_df['doi'].unique()) != 1:
        print(dup_df['pubmed_id'].unique())
        print(dup_df['doi'].unique())

In [16]:
has_pdfpar_df = meta_df[meta_df['pdf_json_files'].isna() == False]
pdfpar_dup = has_pdfpar_df[has_pdfpar_df.duplicated(['pdf_json_files'])]
print("Duplicte pdf paths:", len(pdfpar_dup))
print("Total pdf paths", len(has_pdfpar_df))

Duplicte pdf paths: 13
Total pdf paths 48924


In [17]:
has_pmcpar_df = meta_df[meta_df['pmc_json_files'].isna() == False]
pmcpar_dup = has_pmcpar_df[has_pmcpar_df.duplicated(['pmc_json_files'])]
print("Duplicte pmc paths:", len(pmcpar_dup))
print("Total pmc paths", len(has_pmcpar_df))

Duplicte pmc paths: 0
Total pmc paths 33503


In [31]:
has_sha_df = meta_df[meta_df['sha'].isna() == False]
mult_sha_df = has_sha_df[has_sha_df['sha'].str.len() != 40]
sing_sha_df = has_sha_df[has_sha_df['sha'].str.len() == 40]
print("# of records with multiple associated sha values:", len(mult_sha_df))

# of records with multiple associated sha values: 2517


In [30]:
has_pmc_df = meta_df[meta_df['pmcid'].isna() == False]
print("# of weird pmcid values:", len(has_pmc_df[(has_pmc_df['pmcid'].str.len() != 9) & (has_pmc_df['pmcid'].str.len() != 10) & (has_pmc_df['pmcid'].str.len() != 8)]))

# of weird pmcid values: 0


In [23]:
from os import listdir
PARSE_PATH ="/".join([DATAPATH, VERSION, 'document_parses'])

In [26]:
pdf_path = "/".join([PARSE_PATH, 'pdf_json'])
pmc_path = "/".join([PARSE_PATH, 'pmc_json'])
pdf_parses = listdir(pdf_path)
pmc_parses = listdir(pmc_path)
print("Total # pdfs:", len(pmc_parses))
print("Total # pmcs:", len(pdf_parses))

Total # pdfs: 33503
Total # pmcs: 51868


In [34]:
sha_list = []
for row in mult_sha_df.iterrows():
    new_l = row[1].sha.split("; ")
    sha_list.extend(new_l)
print("All sha values in multi sha record list:", len(sha_list))
print("All UNIQUE sha values in multi sha record list:", len(set(sha_list)))

print("All sha values in single sha record list:", len(list(sing_sha_df.sha)))
print("All UNIQUE sha values in single sha record list:", len(set(sing_sha_df.sha)))

sha_list.extend(list(sing_sha_df.sha))
print("All sha values in combined sha record list:", len(sha_list))
print("All UNIQUE sha values in combined sha record list:", len(set(sha_list)))
total_unique_sha = len(set(sha_list))

All sha values in multi sha record list: 5475
All UNIQUE sha values in multi sha record list: 5475
All sha values in single sha record list: 46407
All UNIQUE sha values in single sha record list: 46394
All sha values in combined sha record list: 51882
All UNIQUE sha values in combined sha record list: 51868
